In [3]:
import pandas as pd 
import numpy as np
from langdetect import detect
import seaborn as sns
import io 
#from s3_credentials import *

In [5]:
import boto3

In [11]:
# boto session 
YOUR_ACCESS_KEY = 
YOUR_SECRET_KEY = 

session = boto3.Session(aws_access_key_id= YOUR_ACCESS_KEY, 
                        aws_secret_access_key= YOUR_SECRET_KEY)

s3 = session.resource("s3")
client = session.client("s3")

### Pull datasets

In [12]:
#Pull subdatasets

# real
obj = s3.Object('jedha-fake-reviews-project', "datasets/real_reviews_raw.csv")
real = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), low_memory = False, index_col=0)

# fake
obj = s3.Object('jedha-fake-reviews-project', "datasets/fake_reviews_raw.csv")
fake = pd.read_csv(io.BytesIO(obj.get()['Body'].read()), low_memory = False, index_col=0)


### Merge both datasets

In [39]:
# find order columned list
columns_list_ordered = real.columns.values.tolist()
columns_list_ordered.append('url_not_recommended')

# create mapping table 
pivot_table = real.loc[:, ['url','restaurant_average_rating','restaurant_reviews_count','restaurant_expensiveness','restaurant_name']].drop_duplicates('url')

# merge fake with mapping table
fake_merged = pd.merge(fake, pivot_table, left_on='url', right_on = 'url', how='left', suffixes=('', '_y'))

# fake_dataset_reworked = fake_merged.loc[:, columns_list_ordered] --> line has been depreciated 
fake_dataset_reworked = fake_merged.reindex(columns = columns_list_ordered)


# concat both and put into full_dataset
full_dataset = pd.concat([real, fake_dataset_reworked])
full_dataset = full_dataset.reset_index(drop = True)
full_dataset = full_dataset.drop_duplicates()
full_dataset['url_not_recommended'] = full_dataset['url'].apply(lambda url: url.replace("biz/", "not_recommended_reviews/"))

In [41]:
fake_dataset_reworked

,date,username,photos_for_review,rating,text_review,user_location,user_friends_count,user_reviews_count,user_total_image_posted,restaurant_average_rating,restaurant_reviews_count,restaurant_expensiveness,restaurant_name,url,is_real_review,url_not_recommended
0,21/08/2007,Not_Yelp_User,0,5.0,Bon retour !\nJe suis revenue dans ce resto ap...,"Levallois-Perret, Hauts-de-Seine",0,4,0.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
1,140,Not_Yelp_User,0,1.0,"Très déçue!\n\nDes erreurs dans les commandes,...",Paris,11,250,140.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
2,26/07/2006,Benjamin D.,0,3.0,A optimiser...\nCuisine très traditionnelle da...,"Bron, Rhône",0,22,0.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
3,14/10/2004,Not_Yelp_User,0,3.0,Brasserie chic\nUne brasserie authentiquement ...,Marseille,11,155,0.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
4,11/02/2007,Not_Yelp_User,0,4.0,Tres bien\nPetit diner entre amis. Les plats e...,"Boulogne-Billancourt, Hauts-de-Seine",0,10,0.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90116,27/03/2015,Coralie J.,-1.0,1.0,Les serveurs sont d'une nonchalence jamais vue...,Paris,9,3,NaN,2.5,28.0,2,Indiana Café,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...
90117,12/03/2015,Inès ..,-1.0,1.0,J'avais dans l'idée de commander un fish and c...,"Ivry-sur-Seine, Val-de-Marne",0,1,NaN,2.5,28.0,2,Indiana Café,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...
90118,28/11/2011,Not Yelp User,-1.0,1.0,A éviter d'urgence. J'y suis allée samedi en f...,Paris,0,2,1.0,2.5,28.0,2,Indiana Café,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...
90119,17/04/2011,Not Yelp User,-1.0,1.0,Je n'ai jamais vu un accueil aussi déplorable....,"St-Malo, Ille-et-Vilaine",1,4,NaN,2.5,28.0,2,Indiana Café,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...


In [25]:
fake_dataset_reworked

,date,username,photos_for_review,rating,text_review,user_location,user_friends_count,user_reviews_count,user_total_image_posted,restaurant_average_rating,restaurant_reviews_count,restaurant_expensiveness,restaurant_name,url,is_real_review,url_not_recommended
0,21/08/2007,Not_Yelp_User,0,5.0,Bon retour !\nJe suis revenue dans ce resto ap...,"Levallois-Perret, Hauts-de-Seine",0,4,0.0,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
1,140,Not_Yelp_User,0,1.0,"Très déçue!\n\nDes erreurs dans les commandes,...",Paris,11,250,140.0,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
2,26/07/2006,Benjamin D.,0,3.0,A optimiser...\nCuisine très traditionnelle da...,"Bron, Rhône",0,22,0.0,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
3,14/10/2004,Not_Yelp_User,0,3.0,Brasserie chic\nUne brasserie authentiquement ...,Marseille,11,155,0.0,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
4,11/02/2007,Not_Yelp_User,0,4.0,Tres bien\nPetit diner entre amis. Les plats e...,"Boulogne-Billancourt, Hauts-de-Seine",0,10,0.0,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90116,27/03/2015,Coralie J.,-1.0,1.0,Les serveurs sont d'une nonchalence jamais vue...,Paris,9,3,NaN,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...
90117,12/03/2015,Inès ..,-1.0,1.0,J'avais dans l'idée de commander un fish and c...,"Ivry-sur-Seine, Val-de-Marne",0,1,NaN,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...
90118,28/11/2011,Not Yelp User,-1.0,1.0,A éviter d'urgence. J'y suis allée samedi en f...,Paris,0,2,1.0,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...
90119,17/04/2011,Not Yelp User,-1.0,1.0,Je n'ai jamais vu un accueil aussi déplorable....,"St-Malo, Ille-et-Vilaine",1,4,NaN,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...


In [24]:
fake_dataset_reworked

,date,username,photos_for_review,rating,text_review,user_location,user_friends_count,user_reviews_count,user_total_image_posted,restaurant_average_rating,restaurant_reviews_count,restaurant_expensiveness,restaurant_name,url,is_real_review,url_not_recommended
0,21/08/2007,Not_Yelp_User,0,5.0,Bon retour !\nJe suis revenue dans ce resto ap...,"Levallois-Perret, Hauts-de-Seine",0,4,0.0,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
1,140,Not_Yelp_User,0,1.0,"Très déçue!\n\nDes erreurs dans les commandes,...",Paris,11,250,140.0,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
2,26/07/2006,Benjamin D.,0,3.0,A optimiser...\nCuisine très traditionnelle da...,"Bron, Rhône",0,22,0.0,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
3,14/10/2004,Not_Yelp_User,0,3.0,Brasserie chic\nUne brasserie authentiquement ...,Marseille,11,155,0.0,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
4,11/02/2007,Not_Yelp_User,0,4.0,Tres bien\nPetit diner entre amis. Les plats e...,"Boulogne-Billancourt, Hauts-de-Seine",0,10,0.0,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90116,27/03/2015,Coralie J.,-1.0,1.0,Les serveurs sont d'une nonchalence jamais vue...,Paris,9,3,NaN,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...
90117,12/03/2015,Inès ..,-1.0,1.0,J'avais dans l'idée de commander un fish and c...,"Ivry-sur-Seine, Val-de-Marne",0,1,NaN,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...
90118,28/11/2011,Not Yelp User,-1.0,1.0,A éviter d'urgence. J'y suis allée samedi en f...,Paris,0,2,1.0,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...
90119,17/04/2011,Not Yelp User,-1.0,1.0,Je n'ai jamais vu un accueil aussi déplorable....,"St-Malo, Ille-et-Vilaine",1,4,NaN,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...


In [21]:
columns_list_ordered

['date',
 'username',
 'photos_for_review',
 'rating',
 'text_review',
 'user_location',
 'user_friends_count',
 'user_reviews_count',
 'user_total_image_posted',
 'restaurant_average_rating',
 'restaurant_reviews_count',
 'restaurant_expensiveness',
 'restaurant_name',
 'url',
 'is_real_review',
 'url_not_recommended']

In [18]:
full_dataset

,date,username,photos_for_review,rating,text_review,user_location,user_friends_count,user_reviews_count,user_total_image_posted,restaurant_average_rating,restaurant_reviews_count,restaurant_expensiveness,restaurant_name,url,is_real_review,url_not_recommended
0,21/08/2007,Not_Yelp_User,0,5.0,Bon retour !\nJe suis revenue dans ce resto ap...,"Levallois-Perret, Hauts-de-Seine",0,4,0.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
1,140,Not_Yelp_User,0,1.0,"Très déçue!\n\nDes erreurs dans les commandes,...",Paris,11,250,140.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
2,26/07/2006,Benjamin D.,0,3.0,A optimiser...\nCuisine très traditionnelle da...,"Bron, Rhône",0,22,0.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
3,14/10/2004,Not_Yelp_User,0,3.0,Brasserie chic\nUne brasserie authentiquement ...,Marseille,11,155,0.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
4,11/02/2007,Not_Yelp_User,0,4.0,Tres bien\nPetit diner entre amis. Les plats e...,"Boulogne-Billancourt, Hauts-de-Seine",0,10,0.0,3.0,19.0,14,restaurant anatole,https://www.yelp.fr/biz/restaurant-anatole-lev...,1.0,https://www.yelp.fr/not_recommended_reviews/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172839,27/03/2015,Coralie J.,-1.0,1.0,Les serveurs sont d'une nonchalence jamais vue...,Paris,9,3,NaN,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...
172840,12/03/2015,Inès ..,-1.0,1.0,J'avais dans l'idée de commander un fish and c...,"Ivry-sur-Seine, Val-de-Marne",0,1,NaN,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...
172841,28/11/2011,Not Yelp User,-1.0,1.0,A éviter d'urgence. J'y suis allée samedi en f...,Paris,0,2,1.0,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...
172842,17/04/2011,Not Yelp User,-1.0,1.0,Je n'ai jamais vu un accueil aussi déplorable....,"St-Malo, Ille-et-Vilaine",1,4,NaN,NaN,NaN,NaN,NaN,https://www.yelp.fr/biz/indiana-caf%C3%A9-pari...,0.0,https://www.yelp.fr/not_recommended_reviews/in...


### (Optional) Function to return language for each 

In [ ]:
from langdetect import detect
# def function 
def get_lang(x):

    try:
        lang = detect(str(x))
    except:
        lang = 'not_found'

    return lang

In [ ]:
# apply to df 
full_dataset['language'] = full_dataset["text_review"].apply(lambda x : get_lang(x))

### Push to bucket

In [ ]:
# upload new dataset 
# set path and bucket name
PATH = "datasets/full_dataset.csv"
bucket = s3.Bucket(name = "jedha-fake-reviews-project")
# export dataset as csv
data = full_dataset.to_csv()
put_object = bucket.put_object(ACL='private', Key= PATH, Body=data)
#check 
for obj in bucket.objects.all():
    print(obj.key)